In [1]:
import os
import yaml
import json
import pandas as pd

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
import wombat.bsub as bsub

In [7]:
root = '/diskmnt/Projects/Users/estorrs/pecgs-pipeline'

load sample data

In [13]:
input_vcf = '/storage1/fs1/dinglab/Active/Projects/estorrs/sandbox/charger/C3L-00081/HotspotFiltered.vcf.gz'

inheritance_gene_list = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/charger/CharGer_dependencyFiles/PanCan/cancer_pred_genes_160genes_011321_curated_forCharGer.txt'
pp2_gene_list = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/charger/CharGer_dependencyFiles/PanCan/160cpgs.txt'
pathogenic_variants = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/charger/CharGer_dependencyFiles/PanCan/emptyRemoved_20160428_pathogenic_variants_HGVSg_VEP_grch38lifOver.vcf'
hotspot3d_clusters = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/charger/CharGer_dependencyFiles/PanCan/cptac_mc3_combined_noHypers_sorted.maf.3D_Proximity.pairwise.recurrence.l0.ad10.r10.clusters'
clinvar_alleles = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/charger/CharGer_dependencyFiles/clinvar_alleles.single.b38.tsv.gz'

sample = 'C3L-00081'


make test input yaml

In [10]:
template = yaml.safe_load(open(f'{root}/submodules/pecgs-charger/cwl/template.charger.yaml'))
template


{'clinvar_alleles': {'class': 'File', 'path': 'a/file/path'},
 'hotspot3d_clusters': {'class': 'File', 'path': 'a/file/path'},
 'inheritance_gene_list': {'class': 'File', 'path': 'a/file/path'},
 'pathogenic_variants': {'class': 'File', 'path': 'a/file/path'},
 'pp2_gene_list': {'class': 'File', 'path': 'a/file/path'},
 'sample': 'a_string',
 'vcf': {'class': 'File', 'path': 'a/file/path'}}

In [11]:
template['vcf']['path'] = input_vcf
template['inheritance_gene_list']['path'] = inheritance_gene_list
template['pp2_gene_list']['path'] = pp2_gene_list
template['pathogenic_variants']['path'] = pathogenic_variants
template['hotspot3d_clusters']['path'] = hotspot3d_clusters
template['clinvar_alleles']['path'] = clinvar_alleles

template['sample'] = sample

In [12]:
template

{'clinvar_alleles': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/charger/CharGer_dependencyFiles/clinvar_alleles.single.b38.tsv.gz'},
 'hotspot3d_clusters': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/charger/CharGer_dependencyFiles/PanCan/cptac_mc3_combined_noHypers_sorted.maf.3D_Proximity.pairwise.recurrence.l0.ad10.r10.clusters'},
 'inheritance_gene_list': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/charger/CharGer_dependencyFiles/PanCan/cancer_pred_genes_160genes_011321_curated_forCharGer.txt'},
 'pathogenic_variants': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/charger/CharGer_dependencyFiles/PanCan/emptyRemoved_20160428_pathogenic_variants_HGVSg_VEP_grch38lifOver.vcf'},
 'pp2_gene_list': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs_resources/charger/CharGer_dep

In [17]:
yaml.safe_dump(template, open(f'../tests/data/charger/{sample}_inputs.yaml', 'w'))

make cromwell compute1 config

In [18]:
args = bsub.DEFAULT_ARGS
args['group_name'] = 'test_cromwell'
args

{'mem': 10,
 'n_processes': 1,
 'max_mem': None,
 'docker': 'python:3.8',
 'queue': 'dinglab',
 'group': 'compute-dinglab',
 'group_name': 'test_cromwell',
 'n_concurrent': 10,
 'interactive': False,
 'username': 'estorrs'}

In [19]:
workflow_root = f'/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/charger/{sample}'
output_fp = f'../tests/data/charger/{sample}.cromwell-config-db.compute1.dat'
args['queue'] = 'general'
bsub.save_compute1_cromwell_template(workflow_root, output_fp, queue=args['queue'])

make execution script

In [31]:
TEST_DIR_ROOT = '/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/charger'
TOOL_ROOT = '/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline'

dconfig = os.path.join(TEST_DIR_ROOT, f'{sample}.cromwell-config-db.compute1.dat')
cwl_fp = os.path.join(TOOL_ROOT, 'submodules', 'pecgs-charger', 'cwl', 'charger.cwl')
inputs_fp = os.path.join(TEST_DIR_ROOT, f'{sample}_inputs.yaml')
log_fp = os.path.join(TEST_DIR_ROOT, f'{sample}.log')
volumes = [TEST_DIR_ROOT, TOOL_ROOT, workflow_root, '/storage1/fs1/dinglab/Active', '/storage1/fs1/m.wyczalkowski/Active', '/scratch1/fs1/dinglab/estorrs']

In [36]:
run_commands = bsub.cromwell_commands(dconfig, cwl_fp, inputs_fp, args, volumes,
                                     workflow_root=workflow_root, interactive=False,
                                     log_fp=log_fp)
filepath = os.path.join('../tests/data/charger/run_job.sh')
bsub.write_command_file(run_commands, filepath)

In [37]:
run_commands

['mkdir -p /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/charger/C3L-00081/cromwell-workdir/logs',
 'source /opt/ibm/lsfsuite/lsf/conf/lsf.conf',
 'export LSF_DOCKER_NETWORK=host',
 'export LSF_DOCKER_VOLUMES="/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/charger:/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/charger /storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline:/storage1/fs1/dinglab/Active/Projects/estorrs/pecgs-pipeline /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/charger/C3L-00081:/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/charger/C3L-00081 /storage1/fs1/dinglab/Active:/storage1/fs1/dinglab/Active /storage1/fs1/m.wyczalkowski/Active:/storage1/fs1/m.wyczalkowski/Active /scratch1/fs1/dinglab/estorrs:/scratch1/fs1/dinglab/estorrs"',
 'bgadd -L 10 /estorrs/test_cromwell',
 'export PATH="/opt/java/openjdk/bin:$PATH"',
 "bsub -n 1 -q general -G compute-dinglab -a 'docker(estorrs/cromwell-runner:58)' 